$$ \LARGE\bf{EXAM \ PROJECT \ ISDS 2022} $$

$\textbf{Imports}$

In [10]:
import requests
import json
import numbers
import pandas as pd
import time
import tqdm
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

$\textbf{Data}$, housing prices boliga.dk - ONLY FOR FIRST PAGE

In [11]:
#1. Load page boliga.dk
links = []
for page in range(1,147,1):
    url_boliga = f'https://www.boliga.dk/resultat?propertyType=3&page={page}'
    links.append(url_boliga)

In [14]:
list_htmls = []
for url_boliga in tqdm.tqdm(links): #Track the time left before completing the loop
    response_boliga = requests.get(url_boliga, headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
    html = response.text
    list_htmls.append(html)
    time.sleep(0.5) #Sleep for 0.5 seconds

soup = BeautifulSoup(response_boliga.content,"lxml")

#2. Scrape housing prices from www.boliga.dk
list_price = soup.find_all("div" ,class_ = "price")
price = [col.text.split(u'\xa0')[0] for col in list_price]

100%|██████████████████████████████████████████| 146/146 [04:44<00:00,  1.95s/it]


$\bf{Data}$, address, zipcode from boliga.dk -  ONLY FOR FIRST PAGE

In [15]:
#1. Scrape address from www.boliga.dk (first page)
list_address = soup.find_all("div" ,class_ = 'w-100 mt-1 listing-address font-weight-bold text-nowrap text-truncate')
address = [col.text.split(',',1)[0] for col in list_address]

#2. Scrape zipcode from www.boliga.dk (first page)
list_zipcode = soup.find_all("div" ,class_ = "zip-color text-nowrap text-truncate")
zipcode = [col.text for col in list_zipcode]

$\textbf{Data}$, m2, rooms from boliga.dk -  ONLY FOR FIRST PAGE

In [20]:
#1. Scrape size of houses
list_m2 = soup.find_all("span" ,class_ = "text-nowrap")
m2 = [col.text.split() for col in list_m2]

#the variable m2 is a list of lists. The following code keeps the first element of every sublist to get rid of the string "m2"
m2 = [element[0] for element in m2]

del m2[0] #delete first element in m2 in order to get all m2 in next code
m2 = m2[::6] #we now keep every 6th element, which is the size of every address

#2. Scrape number of rooms and prepare
list_rooms = soup.find_all("span" ,class_ = "text-nowrap")
rooms = [col.text.split() for col in list_rooms]
rooms = rooms[::6]
rooms = [element[1] for element in rooms]
rooms

for i in range(len(rooms)):
    if rooms[i] == 'værelse':
        rooms[i] = '1'

$\bf{Combine \ data \ in \ one \ dataset}$

In [21]:
#1. Combine all scraped lists in one dataframe
df_addzip = {"Price": price, "Address": address, "Zipcode": zipcode, "m2": m2, "rooms" : rooms}
df_data = pd.DataFrame(df_addzip)
#df_data

$\bf{Use \ combined \ dataset \ to \ get \ x,y \ coordinates}$, distance to nearest (daycare eksempel) 

In [22]:
#1. Get longitude and latitude of each address in df_data["Address"]
z = []

for a in df_data["Address"]:    
    response_dist = requests.get(f"https://api.dataforsyningen.dk/adresser?q={a}&struktur=mini",headers={'name':'Sevda Öztürk','email':'txg194@alumni.ku.dk'})
    res_j = response_dist.json()
    z.append(res_j)

#2 Get longitude, x, and latitudes, y from address i
length = list(range(len(z)))
x = []
y = []

for i in length:
    xx = z[i][0]["x"]
    x.append(xx)
    
for i in length:
    yy = z[i][0]["y"]
    y.append(yy)

In [24]:
#3. Add x and y to dataset df_data
df_data["x"], df_data["y"] = x, y
#df_data["yx"] = df_data['y'] + ',' + df_data['x']
print(df_data.to_string())

         Price                Address           Zipcode   m2 rooms          x          y
0      385.000            Smedegade 3     6430 Nordborg   97     3   9.922129  57.059276
1      499.000        Solvangsvej 7A         9640 Farsø   89     4   9.345420  56.775994
2      845.000        Solbakkevej 15   9230 Svenstrup J   91     5  11.568835  55.414591
3      335.000              Elmevej 3       7600 Struer   51     2   8.306360  56.195721
4      335.000              Elmevej 3       7600 Struer   51     2   8.306360  56.195721
5    1.050.000          Gl Færgevej 5       7600 Struer   94     3   8.434670  55.471461
6    2.900.000          Strandgade 12         9240 Nibe  178     3  10.528456  57.332983
7      499.000            Skelvej 6B      8981 Spentrup   54     2  10.041683  56.537981
8      645.000              Elmevej 6       7600 Struer  105     4   8.515213  55.903621
9      195.000           Tranevej 108       9990 Skagen   74     2  11.214091  55.389319
10     695.000       

$\bf{Use \ coordinates \ to \ get \ distance}$

In [9]:
dist = []
for x,y in df_data:
    afstand_daycare_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={df_data["y"]},{df_data["x"]}&poitypes=daycare&decodepolyline')
    afstand_daycare_json.json()


ValueError: too many values to unpack (expected 2)